In [221]:
import pandas as pd
import glob
from datetime import datetime
import csv

In [222]:
c_path = "C:\\Users\\ASUS\\OneDrive\\Máy tính\\KHTN_HK2\\Covid_project\\"

In [223]:
def extract_csv(load_file):
    dataframe = pd.read_csv(load_file)
    return dataframe

In [224]:
path_csv_position           = c_path + "datasource\\position\\"
path_csv_time_series        = c_path + "datasource\\time_series"
path_csv_position_Comfirmed = path_csv_position + "time_series_Comfirmed\\"
path_csv_position_Death     = path_csv_position + "time_series_Death\\"
path_csv_position_Recovered = path_csv_position + "time_series_Recovered\\"
path_target                 = c_path + "targetfile\\"
path_target_time_series     = path_target + "time_series_target_file.csv"
path_targer_position_comfirmed = path_target + "position_comfirmed_target_file.csv"
path_targer_position_death = path_target + "position_death_target_file.csv"
path_targer_position_recovered = path_target + "position_recovered_target_file.csv"

In [225]:
def extract(path_file):
    target_raw = []
    for csv_file in glob.glob(path_file + "/*.csv"):
        data = extract_csv(csv_file)
        target_raw.append(data)
    extracted_file = pd.concat(target_raw, ignore_index = True)
    return extracted_file

In [226]:
def load(csv_path, load_data):
    load_data.to_csv(csv_path, index = False)
    return None

In [227]:
def log(message):
    time_format = '%H:%M:%S-%h-%d-%Y'
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(time_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [228]:
def transform_csv_time_series(load_dataframe):
    
    for column in load_dataframe.columns:
        if column != 'Province/State':
            load_dataframe[column].fillna(0, inplace=True)

    date_columns = ['Last Update'] # trans format time to Hour-Minute-Second-MonthName-Day-Year
    for column in date_columns:
        load_dataframe[column] = pd.to_datetime(load_dataframe[column], dayfirst=True).dt.strftime('%H:%M:%S-%h-%d-%Y')
    
    load_dataframe.drop(columns = ['Suspected', 'ConfnSusp', 'Notes'], inplace = True)
    
    # Del row Confirmed, Deaths, Recovered == 0
    #load_dataframe = load_dataframe[~((df['Confirmed'] == 0) & (df['Deaths'] == 0) & (df['Recovered'] == 0))]
    
    load_dataframe.sort_values(by='Last Update', ascending=True, inplace=True)

    return load_dataframe

In [229]:
def read_position_comfirmed():
    df1 = pd.read_csv(path_csv_position_Comfirmed + "time_series_19-covid-Confirmed_archived_0325.csv")
    df2 = pd.read_csv(path_csv_position_Comfirmed + "time_series_2019-ncov-Confirmed.csv")

    set1 = set(df1.columns)
    set2 = set(df2.columns)

    columns = list(set1 & set2)

    for column in columns:
        if column not in df1.columns:
            df1.insert(-1, column, 0)
        if column not in df2.columns:
            df2.insert(-1, column, 0)  
    
    # combine data frame
    combined_df = pd.concat([df1, df2], ignore_index=True)
    dataframe = pd.DataFrame(combined_df)
    
    return dataframe

In [230]:
def read_position_Death():
    df1 = pd.read_csv(path_csv_position_Death + "time_series_19-covid-Deaths_archived_0325.csv")
    df2 = pd.read_csv(path_csv_position_Death + "time_series_2019-ncov-Deaths.csv")

    set1 = set(df1.columns)
    set2 = set(df2.columns)

    columns = list(set1 & set2)

    for column in columns:
        if column not in df1.columns:
            df1.insert(-1, column, 0)
        if column not in df2.columns:
            df2.insert(-1, column, 0)  
    
    # combine data frame
    combined_df = pd.concat([df1, df2], ignore_index=True)
    dataframe = pd.DataFrame(combined_df)
    
    return dataframe

In [231]:
def read_position_Recovered():
    df1 = pd.read_csv(path_csv_position_Recovered + "time_series_19-covid-Recovered_archived_0325.csv")
    df2 = pd.read_csv(path_csv_position_Recovered + "time_series_2019-ncov-Recovered.csv")

    set1 = set(df1.columns)
    set2 = set(df2.columns)

    columns = list(set1 & set2)

    for column in columns:
        if column not in df1.columns:
            df1.insert(-1, column, 0)
        if column not in df2.columns:
            df2.insert(-1, column, 0)  
    
    # combine data frame
    combined_df = pd.concat([df1, df2], ignore_index=True)
    dataframe = pd.DataFrame(combined_df)
    
    return dataframe

In [232]:
def transform_csv_position(load_dataframe):
    
    for column in load_dataframe.columns:
        if column != 'Province/State':
            load_dataframe[column].fillna(0, inplace=True)
    
    new_column_names = []
    for column in load_dataframe.columns:
        if column not in ['Province/State', 'Country/Region', 'Lat', 'Long']:
            new_name = pd.to_datetime(column, dayfirst=True).strftime('%H:%M:%S-%h-%d-%Y')
            new_column_names.append(new_name)
        else:
            new_column_names.append(column)
    load_dataframe.columns = new_column_names
    
    column_labels = load_dataframe.columns.tolist()
    filtered_labels = [label for label in column_labels if label not in ['Province/State', 'Country/Region', 'Lat', 'Long']]

    sorted_df = load_dataframe[filtered_labels].sort_index(axis=1)
    
    return load_dataframe

In [233]:
log("Extract file time series csv")
data_time_series = extract(path_csv_time_series)
df_time_series = pd.DataFrame(data_time_series)
log("Extracted file time series csv")

log("Transform time series csv file")
trans_file_time_series = transform_csv_time_series(df_time_series)
log("Transformed time series csv file")

log("Load time series csv file")
load(path_target_time_series, trans_file_time_series)
log("Loaded time series csv file")

log("Etract file position comfirmed csv")
data_pos_comfirmed = extract(path_csv_position_Comfirmed)
log("Extracted file position comfirmed csv")

log("Extract file position death csv")
data_pos_death = extract(path_csv_position_Death)
log("Extracted file position death csv")

log("Extract file position revovered csv")
data_pos_recovered = extract(path_csv_position_Recovered)
log("Extracted file position recovered csv")

log("Transform position comfirmed csv file")
df_pos_comfirmed = pd.DataFrame(data_pos_comfirmed)
trans_pos_comfirmed = transform_csv_position(df_pos_comfirmed)
log("Transformed position comfirmed csv file")

log("Transform position death csv file")
df_pos_death = pd.DataFrame(data_pos_death)
trans_pos_death = transform_csv_position(df_pos_death)
log("Transformed position death csv file")

log("Transform position recovered csv file")
df_pos_recovered = pd.DataFrame(data_pos_recovered)
trans_pos_recovered = transform_csv_position(df_pos_recovered)
log("Transformed position recovered csv file")

log("Load position confirmed csv file")
load(path_targer_position_comfirmed, trans_pos_comfirmed)
log("Loaded position confirmed csv file")

log("Load position death csv file")
load(path_targer_position_death, trans_pos_death)
log("Loaded position death csv file")

log("Load position recovered csv file")
load(path_targer_position_recovered, trans_pos_recovered)
log("Loaded position recovered csv file")